In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import normalize
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from numpy import mean
from numpy import absolute
from numpy import sqrt

In [2]:
df = pd.read_csv('data.csv')
df

,DateTime,Births per 1000 People,indicator corruption index,Home Ownership Rate,GDP per Capita,Average Monthly Wage,Consumer Spending,Youth Unemployment Rate,Unemployment Rate,Personal Savings,Labor Force Participation Rate,Inflation Rate,Housing Index,Food Inflation,Consumer Price Index CPI
0,1999-12-31T00:00:00,9.662,77,48.2,25237.48,11368.75,240226.50,7.56,6.25,3.75,61.37,-3.94,54.84,-1.83,73.64
1,2000-12-31T00:00:00,9.355,77,51.1,26933.06,11357.00,251074.50,5.96,5.11,4.47,61.32,-3.73,48.99,-2.18,70.90
2,2001-12-31T00:00:00,9.049,79,52.2,26885.24,11334.75,254590.50,5.97,4.93,2.17,61.48,-1.60,42.15,-0.81,69.76
3,2002-12-31T00:00:00,8.742,82,53.0,27209.87,10965.00,251926.50,8.17,7.22,0.14,61.79,-3.05,38.02,-2.08,67.63
4,2003-12-31T00:00:00,8.436,80,53.4,28096.90,10807.25,248003.25,8.80,7.90,0.00,61.50,-2.60,33.50,-1.60,65.90
5,2004-12-31T00:00:00,8.558,80,54.3,30304.08,10493.50,265679.25,6.98,6.88,0.02,61.32,-0.43,45.61,1.08,65.64
6,2005-12-31T00:00:00,8.680,83,53.7,32401.14,10532.00,275093.75,6.24,5.70,0.97,61.03,0.98,53.35,1.75,66.24
7,2006-12-31T00:00:00,8.802,83,53.5,34457.76,10956.00,291862.25,5.65,4.84,2.50,61.22,2.06,52.64,1.76,67.57
8,2007-12-31T00:00:00,8.924,83,53.1,36371.39,11369.00,317103.00,4.75,4.12,2.10,61.26,2.01,56.65,4.26,68.94
9,2008-12-31T00:00:00,9.046,81,53.6,36923.80,11375.00,323126.25,4.50,3.50,0.10,60.90,4.30,67.30,10.20,71.90


In [3]:
df.columns

Index(['DateTime', 'Births per 1000 People', 'indicator corruption index',
       'Home Ownership Rate', 'GDP per Capita', 'Average Monthly Wage',
       'Consumer Spending', 'Youth Unemployment Rate', 'Unemployment Rate',
       'Personal Savings', 'Labor Force Participation Rate', 'Inflation Rate',
       'Housing Index', 'Food Inflation', 'Consumer Price Index CPI'],
      dtype='object')

In [4]:
matrix_x = df.loc[:, ['indicator corruption index',
       'Home Ownership Rate', 'GDP per Capita', 'Average Monthly Wage',
       'Consumer Spending', 'Youth Unemployment Rate', 'Unemployment Rate',
       'Personal Savings', 'Labor Force Participation Rate', 'Inflation Rate',
       'Housing Index', 'Food Inflation', 'Consumer Price Index CPI']]
matrix_x

,indicator corruption index,Home Ownership Rate,GDP per Capita,Average Monthly Wage,Consumer Spending,Youth Unemployment Rate,Unemployment Rate,Personal Savings,Labor Force Participation Rate,Inflation Rate,Housing Index,Food Inflation,Consumer Price Index CPI
0,77,48.2,25237.48,11368.75,240226.50,7.56,6.25,3.75,61.37,-3.94,54.84,-1.83,73.64
1,77,51.1,26933.06,11357.00,251074.50,5.96,5.11,4.47,61.32,-3.73,48.99,-2.18,70.90
2,79,52.2,26885.24,11334.75,254590.50,5.97,4.93,2.17,61.48,-1.60,42.15,-0.81,69.76
3,82,53.0,27209.87,10965.00,251926.50,8.17,7.22,0.14,61.79,-3.05,38.02,-2.08,67.63
4,80,53.4,28096.90,10807.25,248003.25,8.80,7.90,0.00,61.50,-2.60,33.50,-1.60,65.90
5,80,54.3,30304.08,10493.50,265679.25,6.98,6.88,0.02,61.32,-0.43,45.61,1.08,65.64
6,83,53.7,32401.14,10532.00,275093.75,6.24,5.70,0.97,61.03,0.98,53.35,1.75,66.24
7,83,53.5,34457.76,10956.00,291862.25,5.65,4.84,2.50,61.22,2.06,52.64,1.76,67.57
8,83,53.1,36371.39,11369.00,317103.00,4.75,4.12,2.10,61.26,2.01,56.65,4.26,68.94
9,81,53.6,36923.80,11375.00,323126.25,4.50,3.50,0.10,60.90,4.30,67.30,10.20,71.90


In [5]:
#Min-Max normolisation
df_min_max_scaled = matrix_x.copy()

for column in df_min_max_scaled.columns:
    df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())    
  
print(df_min_max_scaled)
#matrix_normed = pd.DataFrame(df_min_max_scaled).to_numpy()
#matrix_normed

    indicator corruption index  Home Ownership Rate  GDP per Capita  \
0                          0.3             0.000000        0.000000   
1                          0.3             0.475410        0.084598   
2                          0.5             0.655738        0.082212   
3                          0.8             0.786885        0.098409   
4                          0.6             0.852459        0.142666   
5                          0.6             1.000000        0.252790   
6                          0.9             0.901639        0.357419   
7                          0.9             0.868852        0.460031   
8                          0.9             0.803279        0.555509   
9                          0.7             0.885246        0.583070   
10                         0.8             0.868852        0.533903   
11                         1.0             0.803279        0.641244   
12                         1.0             0.819672        0.719392   
13    

In [6]:
x_train = df_min_max_scaled.iloc[:15,:] #1999-2014
x_train

,indicator corruption index,Home Ownership Rate,GDP per Capita,Average Monthly Wage,Consumer Spending,Youth Unemployment Rate,Unemployment Rate,Personal Savings,Labor Force Participation Rate,Inflation Rate,Housing Index,Food Inflation,Consumer Price Index CPI
0,0.3,0.000000,0.000000,0.132318,0.000000,0.711628,0.674556,0.838926,0.804651,0.000000,0.148040,0.028271,0.235433
1,0.3,0.475410,0.084598,0.130542,0.042192,0.339535,0.449704,1.000000,0.781395,0.022777,0.107458,0.000000,0.154797
2,0.5,0.655738,0.082212,0.127178,0.055867,0.341860,0.414201,0.485459,0.855814,0.253796,0.060007,0.110662,0.121248
3,0.8,0.786885,0.098409,0.071280,0.045506,0.853488,0.865878,0.031320,1.000000,0.096529,0.031356,0.008078,0.058564
4,0.6,0.852459,0.142666,0.047432,0.030247,1.000000,1.000000,0.000000,0.865116,0.145336,0.000000,0.046850,0.007652
5,0.6,1.000000,0.252790,0.000000,0.098995,0.576744,0.798817,0.004474,0.781395,0.380694,0.084010,0.263328,0.000000
6,0.9,0.901639,0.357419,0.005820,0.135612,0.404651,0.566075,0.217002,0.646512,0.533623,0.137704,0.317447,0.017657
7,0.9,0.868852,0.460031,0.069919,0.200831,0.267442,0.396450,0.559284,0.734884,0.650759,0.132778,0.318255,0.056798
8,0.9,0.803279,0.555509,0.132356,0.299002,0.058140,0.254438,0.469799,0.753488,0.645336,0.160597,0.520194,0.097116
9,0.7,0.885246,0.583070,0.133263,0.322428,0.000000,0.132150,0.022371,0.586047,0.893709,0.234478,1.000000,0.184226


In [7]:
x_test = df_min_max_scaled.iloc[15:,:]
x_test

,indicator corruption index,Home Ownership Rate,GDP per Capita,Average Monthly Wage,Consumer Spending,Youth Unemployment Rate,Unemployment Rate,Personal Savings,Labor Force Participation Rate,Inflation Rate,Housing Index,Food Inflation,Consumer Price Index CPI
15,0.0,0.442623,0.826190,0.566424,0.695514,0.211628,0.080868,0.002237,0.702326,0.907809,0.608186,0.509693,0.667746
16,0.1,0.344262,0.857888,0.658264,0.773119,0.320930,0.092702,0.002237,0.804651,0.752711,0.721609,0.495153,0.745438
17,0.3,0.360656,0.890587,0.722174,0.807195,0.337209,0.110454,0.002237,0.716279,0.689805,0.687686,0.449111,0.809888
18,0.3,0.196721,0.955112,0.787596,0.902809,0.304651,0.059172,0.002237,0.716279,0.587852,0.846757,0.352181,0.850795
19,0.2,0.196721,1.000000,0.906270,1.000000,0.223256,0.000000,0.008949,0.716279,0.689805,0.990149,0.448304,0.917893
20,0.2,0.311475,0.945724,1.000000,0.984529,0.216279,0.019724,0.024609,0.432558,0.739696,1.000000,0.568659,1.000000


In [8]:
y = df[['Births per 1000 People']]

In [9]:
y_train = y.iloc[:15,:]
y_train

,Births per 1000 People
0,9.662
1,9.355
2,9.049
3,8.742
4,8.436
5,8.558
6,8.680
7,8.802
8,8.924
9,9.046


In [10]:
y_test = y.iloc[15:,:]
y_test

,Births per 1000 People
15,10.605
16,10.725
17,10.845
18,10.965
19,11.085
20,11.027


In [11]:
regr_1 = DecisionTreeRegressor(max_depth = 1) 
regr_2 = DecisionTreeRegressor(max_depth = 2)
regr_3 = DecisionTreeRegressor(max_depth = 3)
regr_4 = DecisionTreeRegressor(max_depth = 4)
regr_5 = DecisionTreeRegressor(max_depth = 5)
regr_6 = DecisionTreeRegressor(max_depth = 6) 
regr_7 = DecisionTreeRegressor(max_depth = 7)
regr_8 = DecisionTreeRegressor(max_depth = 8)
regr_9 = DecisionTreeRegressor(max_depth = 9) 
regr_10 = DecisionTreeRegressor(max_depth = 10)

In [12]:
#max-depth = 1
cv = LeaveOneOut()
X1 = x_train.copy()
X2 = X1.to_numpy()
y = y_train.to_numpy()
y_true, y_pred = list(), list()
myList = list(range(1,11))
for train_ix, test_ix in cv.split(X2):
    # split data
    X_train, X_test = X2[train_ix, :], X2[test_ix, :]
    Y_train, Y_test = y[train_ix], y[test_ix]
    # fit model
    model = regr_1
    model.fit(X_train, Y_train)
    # evaluate model
    yhat = model.predict(X_test)
    y_true.append(Y_test[0])
    y_pred.append(yhat[0])
# calculate accuracy
MSE_1 = mean_squared_error(y_true,y_pred)
print('MSE: %.3f' % MSE_1)

#5 outputs of MSE_1 = 0.134, 0.062, 0.132, 0.062, 0.076 --> Average = 0.0932

MSE: 0.165


In [13]:
#max-depth = 2
cv = LeaveOneOut()
X1 = x_train.copy()
X2 = X1.to_numpy()
y = y_train.to_numpy()
y_true, y_pred = list(), list()
for train_ix, test_ix in cv.split(X2):
    # split data
    X_train, X_test = X2[train_ix, :], X2[test_ix, :]
    Y_train, Y_test = y[train_ix], y[test_ix]
    # fit model
    model = regr_2
    model.fit(X_train, Y_train)
    # evaluate model
    yhat = model.predict(X_test)
    y_true.append(Y_test[0])
    y_pred.append(yhat[0])
# calculate accuracy
MSE_2 = mean_squared_error(y_true,y_pred)
print('MSE: %.3f' % MSE_2)

#5 outputs of MSE_2 = 0.106, 0.073, 0.067, 0.069, 0.101  --> Average = 0.0832

MSE: 0.115


In [14]:
#max-depth = 3
cv = LeaveOneOut()
X1 = x_train.copy()
X2 = X1.to_numpy()
y = y_train.to_numpy()
y_true, y_pred = list(), list()
for train_ix, test_ix in cv.split(X2):
    # split data
    X_train, X_test = X2[train_ix, :], X2[test_ix, :]
    Y_train, Y_test = y[train_ix], y[test_ix]
    # fit model
    model = regr_3
    model.fit(X_train, Y_train)
    # evaluate model
    yhat = model.predict(X_test)
    y_true.append(Y_test[0])
    y_pred.append(yhat[0])
# calculate accuracy
MSE_3 = mean_squared_error(y_true,y_pred)
print('MSE: %.3f' % MSE_3)

#5 outputs of MSE_3 = 0.085, 0.099, 0.101, 0.063, 0.106  --> Average = 0.0696

MSE: 0.107


In [15]:
#max-depth = 4
cv = LeaveOneOut()
X1 = x_train.copy()
X2 = X1.to_numpy()
y = y_train.to_numpy()
y_true, y_pred = list(), list()
for train_ix, test_ix in cv.split(X2):
    # split data
    X_train, X_test = X2[train_ix, :], X2[test_ix, :]
    Y_train, Y_test = y[train_ix], y[test_ix]
    # fit model
    model = regr_4
    model.fit(X_train, Y_train)
    # evaluate model
    yhat = model.predict(X_test)
    y_true.append(Y_test[0])
    y_pred.append(yhat[0])
# calculate accuracy
MSE_4 = mean_squared_error(y_true,y_pred)
print('MSE: %.3f' % MSE_4)

#5 outputs of MSE_4 = 0.081, 0.091, 0.087, 0.072, 0.112 --> Average = 0.0662

MSE: 0.093


In [16]:
#max-depth = 5
cv = LeaveOneOut()
X1 = x_train.copy()
X2 = X1.to_numpy()
y = y_train.to_numpy()
y_true, y_pred = list(), list()
for train_ix, test_ix in cv.split(X2):
    # split data
    X_train, X_test = X2[train_ix, :], X2[test_ix, :]
    Y_train, Y_test = y[train_ix], y[test_ix]
    # fit model
    model = regr_5
    model.fit(X_train, Y_train)
    # evaluate model
    yhat = model.predict(X_test)
    y_true.append(Y_test[0])
    y_pred.append(yhat[0])
# calculate accuracy
MSE_5 = mean_squared_error(y_true,y_pred)
print('MSE: %.3f' % MSE_5)

#5 outputs of MSE_5 = 0.082, 0.116, 0.067, 0.101, 0.118  --> Average = 0.0968

MSE: 0.079


In [17]:
#max-depth = 6
cv = LeaveOneOut()
X1 = x_train.copy()
X2 = X1.to_numpy()
y = y_train.to_numpy()
y_true, y_pred = list(), list()
for train_ix, test_ix in cv.split(X2):
    # split data
    X_train, X_test = X2[train_ix, :], X2[test_ix, :]
    Y_train, Y_test = y[train_ix], y[test_ix]
    # fit model
    model = regr_6
    model.fit(X_train, Y_train)
    # evaluate model
    yhat = model.predict(X_test)
    y_true.append(Y_test[0])
    y_pred.append(yhat[0])
# calculate accuracy
MSE_6 = mean_squared_error(y_true,y_pred)
print('MSE: %.3f' % MSE_6)

#5 outputs of MSE_6 = 0.100, 0.086, 0.114, 0.057, 0.084  --> Average = 0.0882

MSE: 0.113


In [18]:
#max-depth = 7
cv = LeaveOneOut()
X1 = x_train.copy()
X2 = X1.to_numpy()
y = y_train.to_numpy()
y_true, y_pred = list(), list()
for train_ix, test_ix in cv.split(X2):
    # split data
    X_train, X_test = X2[train_ix, :], X2[test_ix, :]
    Y_train, Y_test = y[train_ix], y[test_ix]
    # fit model
    model = regr_7
    model.fit(X_train, Y_train)
    # evaluate model
    yhat = model.predict(X_test)
    y_true.append(Y_test[0])
    y_pred.append(yhat[0])
# calculate accuracy
MSE_7 = mean_squared_error(y_true,y_pred)
print('MSE: %.3f' % MSE_7)

#5 outputs of MSE_7 = 0.069, 0.113, 0.067, 0.096, 0.113  --> Average = 0.0916

MSE: 0.099


In [19]:
#max-depth = 8
cv = LeaveOneOut()
X1 = x_train.copy()
X2 = X1.to_numpy()
y = y_train.to_numpy()
y_true, y_pred = list(), list()
for train_ix, test_ix in cv.split(X2):
    # split data
    X_train, X_test = X2[train_ix, :], X2[test_ix, :]
    Y_train, Y_test = y[train_ix], y[test_ix]
    # fit model
    model = regr_8
    model.fit(X_train, Y_train)
    # evaluate model
    yhat = model.predict(X_test)
    y_true.append(Y_test[0])
    y_pred.append(yhat[0])
# calculate accuracy
MSE_8 = mean_squared_error(y_true,y_pred)
print('MSE: %.3f' % MSE_8)

#5 outputs of MSE_4 = 0.114, 0.118, 0.081, 0.117, 0.131 --> Average = 0.1122

MSE: 0.103


In [20]:
#max-depth = 9
cv = LeaveOneOut()
X1 = x_train.copy()
X2 = X1.to_numpy()
y = y_train.to_numpy()
y_true, y_pred = list(), list()
for train_ix, test_ix in cv.split(X2):
    # split data
    X_train, X_test = X2[train_ix, :], X2[test_ix, :]
    Y_train, Y_test = y[train_ix], y[test_ix]
    # fit model
    model = regr_9
    model.fit(X_train, Y_train)
    # evaluate model
    yhat = model.predict(X_test)
    y_true.append(Y_test[0])
    y_pred.append(yhat[0])
# calculate accuracy
MSE_9 = mean_squared_error(y_true,y_pred)
print('MSE: %.3f' % MSE_9)

#5 outputs of MSE_9 = 0.131, 0.073, 0.068, 0.063, 0.129 --> Average = 0.0928

MSE: 0.082


In [21]:
#max-depth = 10
cv = LeaveOneOut()
X1 = x_train.copy()
X2 = X1.to_numpy()
y = y_train.to_numpy()
y_true, y_pred = list(), list()
for train_ix, test_ix in cv.split(X2):
    # split data
    X_train, X_test = X2[train_ix, :], X2[test_ix, :]
    Y_train, Y_test = y[train_ix], y[test_ix]
    # fit model
    model = regr_10
    model.fit(X_train, Y_train)
    # evaluate model
    yhat = model.predict(X_test)
    y_true.append(Y_test[0])
    y_pred.append(yhat[0])
# calculate accuracy
MSE_10 = mean_squared_error(y_true,y_pred)
print('MSE: %.3f' % MSE_10)

#5 outputs of MSE_10 = 0.0928, 0.093, 0.080, 0.103, 0.135  --> Average = 0.10076

MSE: 0.066


# Max-depth = 4 has the smallest MSE

In [23]:
# Train model with Max-depth = 4
regr_ = DecisionTreeRegressor(max_depth=4)
regr_.fit(x_train, y_train)

DecisionTreeRegressor(max_depth=4)

In [24]:
y_1 = regr_1.predict(x_test.values) 
y_1

array([9.7448, 9.7448, 9.7448, 9.7448, 9.7448, 9.7448])

In [25]:
MSE = mean_squared_error(y_test, y_1)
MSE

1.3067061733333303